In [1]:
from langchain.memory import ConversationBufferMemory

memory = ConversationBufferMemory()
memory.save_context({"input": "hi"}, {"ouput": "whats up"})

memory.load_memory_variables({})

{'history': 'Human: hi\nAI: whats up'}

我们还可以将历史记录作为消息列表来获取（如果您将其用于聊天模型，这将非常有用）。

In [2]:
memory = ConversationBufferMemory(return_messages=True)
memory.save_context({"input": "hi"}, {"ouput": "whats up"})

memory.load_memory_variables({})

{'history': [HumanMessage(content='hi', additional_kwargs={}),
  AIMessage(content='whats up', additional_kwargs={})]}

在链条中使用
最后，让我们看看如何在链中使用它（设置verbose=True所以我们可以看到提示）。

In [ ]:
from langchain.llms import OpenAI
from langchain.chains import ConversationChain


llm = OpenAI(temperature=0)
conversation = ConversationChain(
    llm=llm, 
    verbose=True, 
    memory=ConversationBufferMemory()
)



In [ ]:
conversation.predict(input="你好，兄弟!")

In [ ]:
import requests
from bs4 import BeautifulSoup


def search_baidu(keyword):
    url = f"https://www.baidu.com/s?wd={keyword}"
    headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    html = requests.get(url, headers=headers).text
    soup = BeautifulSoup(html, "html.parser")
    results = soup.find_all('div', {'class': 'result'})
    data = []
    for result in results:
        try:
            title = result.h3.a.text.strip()
            summary = result.find('div', {'class': 'c-abstract'}).text.strip()
            data.append({'title': title, 'summary': summary})
        except:
            continue
    return data

In [ ]:
from langchain.agents import ZeroShotAgent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain import OpenAI, LLMChain
from langchain.utilities import GoogleSearchAPIWrapper

search = GoogleSearchAPIWrapper()
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="当你需要回答关于当前时间的问题时，这个工具会对你很有用。"
    )
]


prefix = """与人类交谈, 尽你所能回答以下问题. 你可以使用以下工具:"""
suffix = """开始!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools, 
    prefix=prefix, 
    suffix=suffix, 
    input_variables=["input", "chat_history", "agent_scratchpad"]
)
memory = ConversationBufferMemory(memory_key="chat_history")


llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(agent=agent, tools=tools, verbose=True, memory=memory)

agent_chain.run(input="加拿大有多少人口?")


agent_chain.run(input="加拿大的国歌是什么?")



In [ ]:
! wget https://raw.githubusercontent.com/openai/openai-openapi/master/openapi.yaml
! mv openapi.yaml openai_openapi.yaml
! wget https://www.klarna.com/us/shopping/public/openai/v0/api-docs
! mv api-docs klarna_openapi.yaml
! wget https://raw.githubusercontent.com/APIs-guru/openapi-directory/main/APIs/spotify.com/1.0.0/openapi.yaml
! mv openapi.yaml spotify_openapi.yaml